# toocoolforschool

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import time
import copy

In [2]:
def getNumber(string):
    numExtracter = re.compile('[0-9]+')
    return int(''.join(numExtracter.findall(string)))
    

In [3]:
def getCategoryList():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    categories = soup.find('ul',{'class':'menulist'}).find_all('li') # 사이트마다 다름
    categoryList=[]
    for category in categories:
        if '신제품' in category.get_text():
            continue
        if '베스트' in category.get_text():
            continue
        categoryList.append(url_home + category.a['href'])
    return categoryList

In [4]:
def getItemList():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    items = soup.find('div',{'class':'product_line'}).ul.find_all('li') # 사이트마다 다름
    itemList=[]
    for item in items:
        itemList.append(url_home + '/shop'+item.a['href'][2:])

    return itemList


In [5]:
def goToNextPage():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    try:
        nextPage = url_home + soup.find('a',{'class':'next2'})['href']
        driver.get(nextPage)
        return True
    except:
        return False


In [6]:
def getCategoryItemList():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    goto = True
    itemList = []
    while goto:
        itemList += getItemList()
        goto = goToNextPage()
    return itemList

In [7]:
def getItem():
    html = driver.page_source
    soup = bs(html, 'html.parser')
    item = {'name':'#', 'url':'#', 'image':'#', 'color':'#', 'category':'#', 
                   'salePrice':'#', 'originalPrice':'#', 'brand':'#','volume':'#'}
    item['name'] = soup.find('p',{'class':'title_en'}).get_text().strip()
    image = soup.find('img',{'id':'objImg'})['src']

    item['image'] = url_home + '/shop' + image[2:]

    categoryList = soup.find('div',{'id':'path'}).ul.find_all('li')
    category=''
    for c in categoryList:
        if len(category)<1:
            pass
        else:
            category += ' > '
        category += c.get_text().strip()
    item['category'] = category
    
    item['volume'] = soup.find('th',text='용량 및 규격').parent.td.get_text().strip()
    
    price = soup.find('div',{'class':'value'})
    salePrice = getNumber(price.find('input')['value'])
    originalPrice = price.find('del')
    originalPrice = getNumber(originalPrice.get_text()) if originalPrice else salePrice
    item['originalPrice'] = originalPrice
    item['salePrice'] = salePrice
    
    item['brand'] = 'toocoolforschool'
    
    item['url'] = driver.current_url

    colorList = soup.find('select',{'name':'opt[]'})
    items=[]
    if colorList:
        colorList = colorList.find_all('option')
        for c, color in enumerate(colorList):
            if c==0:
                continue
            item_copy = copy.deepcopy(item)
            item_copy['color'] = color.get_text().strip() 
            items.append(item_copy)
    else:
        items.append(item)
    display(items)
    return items

In [8]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


In [9]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

url_home = 'http://www.toocoolforschool.com'
url_products = 'http://www.toocoolforschool.com/shop/goods/goods_list.php?&category=001'

In [10]:
driver.get(url_products)
result= []
categoryList = getCategoryList()
for category in categoryList:
    driver.get(category)
    itemList=[]
    itemList += getCategoryItemList()
    for item in itemList:
        driver.get(item)
        result += getItem()


[{'name': '★20%할인★에그 멜로우 크림',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110242&category=001',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/14964071469l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 18400,
  'originalPrice': 23000,
  'brand': 'toocoolforschool',
  'volume': '50g'}]

[{'name': '★각질 필수템 1위★맥걸리 고두밥 스크럽',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110103&category=001',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1496411879329l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 12000,
  'originalPrice': 12000,
  'brand': 'toocoolforschool',
  'volume': '80ml'}]

[{'name': '룰스 오브 매스틱 IX 인핸서',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110322&category=001',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1531380293291l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 27000,
  'originalPrice': 27000,
  'brand': 'toocoolforschool',
  'volume': '80g'}]

[{'name': '룰스 오브 매스틱 페이셜 토닉',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110324&category=001',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1512007921368l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 18000,
  'originalPrice': 18000,
  'brand': 'toocoolforschool',
  'volume': '120ml'}]

[{'name': '룰스 오브 매스틱 IX 드롭',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110325&category=001',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1512022995721l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 23000,
  'originalPrice': 23000,
  'brand': 'toocoolforschool',
  'volume': '40ml'}]

...

[{'name': '펜슬 전용 샤프너 12mm',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110173&category=007',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1498805030829l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 1000,
  'originalPrice': 1000,
  'brand': 'toocoolforschool',
  'volume': '12mm'}]

[{'name': '★2in1★아티스트 글로잉 듀얼 터치',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110413&category=007',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1510276885842l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 5000,
  'originalPrice': 5000,
  'brand': 'toocoolforschool',
  'volume': '1EA'}]

[{'name': '아트클래스 듀얼 아이브로우 브러쉬',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110461&category=007',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1535679957231l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 8000,
  'originalPrice': 8000,
  'brand': 'toocoolforschool',
  'volume': '0.8 cm X 16 cm'}]

[{'name': '★2in1★아트클래스 듀얼 컨투어 브러쉬',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110483&category=007',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1540513539951l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 15000,
  'originalPrice': 15000,
  'brand': 'toocoolforschool',
  'volume': '1.9cmX3.2cm / 15.8cm'}]

[{'name': '아트클래스 퀵 드라이 브러쉬 클렌징 티슈',
  'url': 'http://www.toocoolforschool.com/shop/goods/goods_view.php?goodsno=14110498&category=007',
  'image': 'http://www.toocoolforschool.com/shop/data/goods/1544507126800l0.jpg',
  'color': '#',
  'category': '홈 > E-SHOP > BRAND LINE',
  'salePrice': 2500,
  'originalPrice': 2500,
  'brand': 'toocoolforschool',
  'volume': '77g_20ea'}]

In [11]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'toocoolforschool.json')